In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from functools import reduce
from typing import *
import tiktoken
import io

### Localizer Modules
from localizers.general_config import *
from localizers.ml_tracker import MLTracker
from localizers.publishing_config import *
from localizers.publishing_localizer import *

In [0]:
%run "./authenticationScript"

In [0]:
def setup_widgets():
    dbutils.widgets.text('RowFingerprint',"")
    dbutils.widgets.text("Timestamp","")
    dbutils.widgets.text("SubmitterEmail","")
    dbutils.widgets.text("DueDate","")
    dbutils.widgets.text('LocType',"")
    dbutils.widgets.text("Game","")
    dbutils.widgets.text("TargetLanguages","")
    dbutils.widgets.text("URL","")
    dbutils.widgets.text("QAFlag","")
    dbutils.widgets.text("Status","")
    dbutils.widgets.text("LastStatusUpdate","")

def get_request():
    return {'Timestamp':dbutils.widgets.get('Timestamp'), 
       'SubmitterEmail':dbutils.widgets.get('SubmitterEmail'),
       'DueDate': dbutils.widgets.get("DueDate"), 
       'Game':  dbutils.widgets.get("Game"),
       'TargetLanguages':  dbutils.widgets.get("TargetLanguages"), 
       'URL': dbutils.widgets.get("URL"), 
       'LocType': dbutils.widgets.get("LocType"), 
       'RowFingerprint': dbutils.widgets.get("RowFingerprint")}

setup_widgets()

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
rqst = get_request()

cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    "qc": {"enabled": True, "max_retries": 5,}}

In [0]:
localizer = PublishingLocalizer(request = rqst, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
results = localizer.run()

In [0]:
#results = {'status': 'SUCCEEDED', 'run_id': 'test_1234'}

In [0]:
dbutils.notebook.exit(json.dumps(results))
